<a href="https://colab.research.google.com/github/danigallegdup/NUS-Stock-Data/blob/main/Stock_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Price Prediction using LSTM
This project aims to predict stock prices using historical data and Long Short-Term Memory (LSTM) neural networks.

## Data Loading and Preprocessing

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


In [4]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Load the dataset (had to make a shortcut from "Shared with Me" to MyDrive)
sp500=pd.read_csv('/content/drive/MyDrive/cs3244_data/ETFs/spy.us.txt')
sp500.drop(columns={'OpenInt','Volume'}, inplace=True)
# Create empty lists to store the new data
dates = []
prices = []
price_types = []

# Iterate through the DataFrame
for index, row in sp500.iterrows():
  date = row['Date']
  open_price = row['Open']
  high_price = row['High']
  low_price = row['Low']
  close_price = row['Close']

  # Append the data to the lists
  dates.extend([date] * 4)  # Repeat the date 4 times
  prices.extend([open_price, high_price, low_price, close_price])
  price_types.extend(['open', 'high', 'low', 'close'])

# Create a new DataFrame
sp500 = pd.DataFrame({'Date': dates, 'Price': prices, 'Type of Price': price_types})

In [6]:
# Group the DataFrame by Date and Type of Price
grouped = sp500.groupby(['Date', 'Type of Price'])

# Calculate the average price for each date and type
avg_prices = grouped['Price'].mean().reset_index()

# Create a dictionary to store the average high and low prices for each date
avg_high_low = {}

# Iterate through the average prices
for index, row in avg_prices.iterrows():
  date = row['Date']
  price_type = row['Type of Price']
  price = row['Price']

  if date not in avg_high_low:
    avg_high_low[date] = {}

  if price_type == 'high' or price_type == 'low':
    avg_high_low[date][price_type] = price

# Iterate through the DataFrame and replace high and low prices with the average
for index, row in sp500.iterrows():
  date = row['Date']
  price_type = row['Type of Price']

  if price_type == 'high' or price_type == 'low' and date in avg_high_low:
    sp500.loc[index, 'Price'] = avg_high_low[date][price_type]


In [7]:
# Create a new DataFrame to store the modified data
new_sp500 = pd.DataFrame(columns=['Date', 'Price', 'Type of Price'])

# Iterate through the DataFrame
i = 0
while i < len(sp500):
  row = sp500.iloc[i]
  if row['Type of Price'] == 'high':
    # Check if the next row exists and is 'low'
    if i + 1 < len(sp500) and sp500.iloc[i + 1]['Type of Price'] == 'low':
      # Calculate the average of the high and low prices
      average_price = (row['Price'] + sp500.iloc[i + 1]['Price']) / 2
      # Add a new row with the average price and label 'average'
      new_sp500 = pd.concat([new_sp500, pd.DataFrame({'Date': [row['Date']], 'Price': [average_price], 'Type of Price': ['average']})], ignore_index=True)
      i += 2  # Skip the next row (low)
    else:
      # If the next row is not 'low', keep the current row as it is
      new_sp500 = pd.concat([new_sp500, pd.DataFrame({'Date': [row['Date']], 'Price': [row['Price']], 'Type of Price': [row['Type of Price']]})], ignore_index=True)
      i += 1
  else:
    # If the current row is not 'high', keep it as it is
    new_sp500 = pd.concat([new_sp500, pd.DataFrame({'Date': [row['Date']], 'Price': [row['Price']], 'Type of Price': [row['Type of Price']]})], ignore_index=True)
    i += 1

# Replace the original sp500 DataFrame with the modified one
sp500 = new_sp500
sp500

<ipython-input-7-90077ebfbdb0>:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_sp500 = pd.concat([new_sp500, pd.DataFrame({'Date': [row['Date']], 'Price': [row['Price']], 'Type of Price': [row['Type of Price']]})], ignore_index=True)


,Date,Price,Type of Price
0,2005-02-25,104.770,open
1,2005-02-25,105.340,average
2,2005-02-25,105.790,close
3,2005-02-28,105.550,open
4,2005-02-28,105.120,average
...,...,...,...
9598,2017-11-09,257.375,average
9599,2017-11-09,258.170,close
9600,2017-11-10,257.730,open
9601,2017-11-10,257.790,average


## Neural Network Architecture

References: To put in our presentation or somewhere later <br>
https://www.analyticsvidhya.com/blog/2021/12/stock-price-prediction-using-lstm/ <br>
https://www.coursera.org/in/articles/types-of-neural-networks

In our project, we decided to utilise Long short-term memory (LSTM) networks. LSTMs are a specialised type of recurrent neural network architecture, specifically designed to remember information over extended periods of time. This makes them the perfect model to use for our time-series data.


Originally use 'Close' price as we deduced it as most relevant and stable data and introduce the least amount of noise. But now I that I think of it that could be the main reason it is overfitting lol

In [8]:
print(sp500) # test case

scaler = MinMaxScaler(feature_range=(-1, 1))
sp500['Price'] = scaler.fit_transform(sp500['Price'].values.reshape(-1, 1))

            Date    Price Type of Price
0     2005-02-25  104.770          open
1     2005-02-25  105.340       average
2     2005-02-25  105.790         close
3     2005-02-28  105.550          open
4     2005-02-28  105.120       average
...          ...      ...           ...
9598  2017-11-09  257.375       average
9599  2017-11-09  258.170         close
9600  2017-11-10  257.730          open
9601  2017-11-10  257.790       average
9602  2017-11-10  258.090         close

[9603 rows x 3 columns]


In [18]:
# Define a PyTorch Dataset class for stock prices
class StockDataset(Dataset):
    def __init__(self, data, seq_length=10):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, index):
        # Get the sequence of prices and the next price as label
        x = self.data[index:index + self.seq_length]
        y = self.data[index + self.seq_length]
        # Reshape x to (seq_length, 1) for LSTM input
        return torch.tensor(x, dtype=torch.float32).view(-1, 1), torch.tensor(y, dtype=torch.float32)

# Create dataset and dataloader
data = sp500['Price'].values
seq_length = 10
dataset = StockDataset(data, seq_length=seq_length)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)



In [25]:
class LSTMStockPredictor(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super(LSTMStockPredictor, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq)
        predictions = self.linear(lstm_out[-1])  # Take the last output
        return predictions




In [26]:
# Initialize the model, loss function, and optimizer
model = LSTMStockPredictor()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [27]:
# Training the LSTM model
epochs = 100
for epoch in range(epochs):
    for seq, labels in dataloader:
        optimizer.zero_grad()
        # Reshape seq to (seq_length, batch_size, input_size)
        seq = seq.view(seq_length, -1, 1)

        # Forward pass
        y_pred = model(seq)

        # Calculate loss
        single_loss = loss_function(y_pred, labels.view(-1, 1))

        # Backpropagation
        single_loss.backward()
        optimizer.step()

    # Print loss every 10 epochs
    if epoch % 10 == 0:
        print(f'Epoch {epoch} Loss: {single_loss.item()}')



Epoch 0 Loss: 0.25181978940963745
Epoch 10 Loss: 0.2332867980003357
Epoch 20 Loss: 0.17449504137039185
Epoch 30 Loss: 0.19652165472507477
Epoch 40 Loss: 0.19373902678489685
Epoch 50 Loss: 0.16432735323905945
Epoch 60 Loss: 0.21906906366348267
Epoch 70 Loss: 0.21038956940174103
Epoch 80 Loss: 0.2518618106842041
Epoch 90 Loss: 0.2096504271030426


In [42]:
model.eval()  # Set the model to evaluation mode

test_predictions = []
test_data = []

with torch.no_grad():
    for seq, labels in dataloader:  # Use your test dataloader if you have separate training and testing sets
        # Initialize the hidden state for the LSTM
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))

        # Ensure the input sequence shape is (seq_length, batch_size, input_size)
        seq = seq.view(seq_length, -1, 1)

        # Get the model's predictions
        y_pred = model(seq)

        # Append each prediction and corresponding label
        for prediction, actual in zip(y_pred, labels):
            test_predictions.append(prediction.item())  # Convert each prediction to a Python scalar
            test_data.append(actual.item())  # Convert each actual label to a scalar


